# 1. install and import dependencise

In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

In [1]:
import mediapipe as mp
import cv2 

In [2]:
# drawing utilities -> mediapipe makes it a whole heap easier to draw the different componets of my model using these drawing utitlies
# 쉽게 말하면 drawing helper
# ex) landmarks
mp_drawing = mp.solutions.drawing_utils


mp_holistic = mp.solutions.holistic

## makes some detections

In [3]:
# feed from webcam - connecting to webcam 
cap = cv2.VideoCapture(0)

cv2.startWindowThread()
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened(): # check wether or not webcam is open
        ret, frame = cap.read() # frame 변수에 webcam 으로 부터 받아온 이미지를 저장한다.
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections - holistic model에 detection 하는 process 함수에 image 전달
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [8]:
results.pose_landmarks.landmark[0]

x: 0.5283795595169067
y: 0.664250910282135
z: -0.8024789094924927
visibility: 0.9998429417610168

#  2. capture landmarks & export to csv

In [9]:
import csv
import os
import numpy as np

In [12]:
num_coords = len(results.pose_landmarks.landmark) + len(results.face_landmarks.landmark)
num_coords

501

In [13]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [15]:
len(landmarks)

2005

In [18]:
landmarks[-1]

'v501'

In [19]:
with open('coords.csv', mode='w', newline='') as f :
    csv_writer = csv.writer(f, delimiter=',' , quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [27]:
class_name = 'Victorious'

In [28]:
# feed from webcam - connecting to webcam 
cap = cv2.VideoCapture(0)

cv2.startWindowThread()
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened(): # check wether or not webcam is open
        ret, frame = cap.read() # frame 변수에 webcam 으로 부터 받아온 이미지를 저장한다.
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections - holistic model에 detection 하는 process 함수에 image 전달
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Export coordinates
        try:
            # extract landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmarks.x, landmarks.y, landmarks.z, landmarks.visibility] for landmarks in pose]).flatten())
            
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmarks.x, landmarks.y, landmarks.z, landmarks.visibility] for landmarks in face]).flatten())
            
            # concatenate pose and face landmarks
            row = pose_row + face_row
            row.insert(0, class_name)
            
            with open('coords.csv', mode='a', newline='') as f :
                csv_writer = csv.writer(f, delimiter=',' , quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
        except:
            pass
        
        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

# 3. Train custom model using scikit learn

### 3.1 Read in collected Data and process

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [30]:
df = pd.read_csv('coords.csv')

In [31]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Happy,0.550375,0.736092,-0.993660,0.999878,0.567038,0.641875,-0.938201,0.999758,0.586254,...,-0.003334,0.0,0.633530,0.620348,0.028560,0.0,0.638629,0.610766,0.029913,0.0
1,Happy,0.544945,0.729819,-1.041569,0.999883,0.564621,0.634338,-0.981414,0.999769,0.583246,...,-0.006421,0.0,0.635459,0.621011,0.023423,0.0,0.640528,0.611718,0.024630,0.0
2,Happy,0.541567,0.723934,-1.105898,0.999886,0.563330,0.628629,-1.045038,0.999778,0.581650,...,-0.008806,0.0,0.630127,0.621795,0.017871,0.0,0.635559,0.612524,0.018705,0.0
3,Happy,0.541607,0.728098,-1.146215,0.999880,0.564138,0.632043,-1.081770,0.999770,0.582141,...,-0.008424,0.0,0.626426,0.622005,0.015757,0.0,0.632255,0.613136,0.016358,0.0
4,Happy,0.540633,0.729595,-1.241273,0.999880,0.564175,0.632645,-1.175990,0.999773,0.582163,...,-0.009060,0.0,0.621938,0.626272,0.013444,0.0,0.627654,0.617958,0.013918,0.0


In [32]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
799,Victorious,0.496425,0.593430,-0.669274,0.999830,0.511742,0.534296,-0.619719,0.999675,0.528450,...,0.003401,0.0,0.574601,0.573254,0.021600,0.0,0.578985,0.565812,0.022490,0.0
800,Victorious,0.503425,0.606916,-0.690603,0.999844,0.520325,0.543547,-0.640947,0.999700,0.537847,...,0.003378,0.0,0.583959,0.591348,0.021250,0.0,0.588272,0.584137,0.022080,0.0
801,Victorious,0.514170,0.616959,-0.753288,0.999857,0.532409,0.551677,-0.693692,0.999724,0.549591,...,0.003247,0.0,0.590335,0.594324,0.021593,0.0,0.594581,0.587967,0.022282,0.0
802,Victorious,0.517924,0.626695,-0.749026,0.999867,0.536497,0.558733,-0.695481,0.999746,0.553766,...,0.002374,0.0,0.592238,0.595384,0.021352,0.0,0.596520,0.588399,0.022088,0.0
803,Victorious,0.521593,0.636341,-0.779266,0.999877,0.540784,0.568680,-0.722484,0.999763,0.557606,...,0.003359,0.0,0.590138,0.594873,0.022242,0.0,0.594417,0.588144,0.022947,0.0


In [33]:
df[df['class'] == 'Victorious']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
548,Victorious,0.540629,0.740647,-1.082761,0.999641,0.559768,0.657577,-1.019324,0.999366,0.576458,...,-0.001236,0.0,0.611704,0.639333,0.020212,0.0,0.616672,0.631266,0.020700,0.0
549,Victorious,0.532592,0.724487,-0.966621,0.999674,0.552366,0.648120,-0.906328,0.999425,0.569807,...,-0.002918,0.0,0.598709,0.628547,0.014533,0.0,0.603755,0.620550,0.014808,0.0
550,Victorious,0.529639,0.710076,-0.980034,0.999704,0.549754,0.636548,-0.915734,0.999476,0.567030,...,-0.001070,0.0,0.594734,0.623135,0.016386,0.0,0.599553,0.615425,0.016779,0.0
551,Victorious,0.527065,0.701527,-0.966408,0.999729,0.546982,0.629209,-0.900890,0.999518,0.564140,...,0.000341,0.0,0.595139,0.620997,0.017982,0.0,0.599845,0.614053,0.018312,0.0
552,Victorious,0.523267,0.701532,-0.964637,0.999748,0.542303,0.630229,-0.901444,0.999549,0.559687,...,0.000971,0.0,0.590714,0.622753,0.017220,0.0,0.595312,0.615986,0.017516,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,Victorious,0.496425,0.593430,-0.669274,0.999830,0.511742,0.534296,-0.619719,0.999675,0.528450,...,0.003401,0.0,0.574601,0.573254,0.021600,0.0,0.578985,0.565812,0.022490,0.0
800,Victorious,0.503425,0.606916,-0.690603,0.999844,0.520325,0.543547,-0.640947,0.999700,0.537847,...,0.003378,0.0,0.583959,0.591348,0.021250,0.0,0.588272,0.584137,0.022080,0.0
801,Victorious,0.514170,0.616959,-0.753288,0.999857,0.532409,0.551677,-0.693692,0.999724,0.549591,...,0.003247,0.0,0.590335,0.594324,0.021593,0.0,0.594581,0.587967,0.022282,0.0
802,Victorious,0.517924,0.626695,-0.749026,0.999867,0.536497,0.558733,-0.695481,0.999746,0.553766,...,0.002374,0.0,0.592238,0.595384,0.021352,0.0,0.596520,0.588399,0.022088,0.0


In [34]:
x = df.drop(['class'], axis=1) # features
y = df['class'] # target value

In [35]:
x

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,0.550375,0.736092,-0.993660,0.999878,0.567038,0.641875,-0.938201,0.999758,0.586254,0.641840,...,-0.003334,0.0,0.633530,0.620348,0.028560,0.0,0.638629,0.610766,0.029913,0.0
1,0.544945,0.729819,-1.041569,0.999883,0.564621,0.634338,-0.981414,0.999769,0.583246,0.633666,...,-0.006421,0.0,0.635459,0.621011,0.023423,0.0,0.640528,0.611718,0.024630,0.0
2,0.541567,0.723934,-1.105898,0.999886,0.563330,0.628629,-1.045038,0.999778,0.581650,0.628154,...,-0.008806,0.0,0.630127,0.621795,0.017871,0.0,0.635559,0.612524,0.018705,0.0
3,0.541607,0.728098,-1.146215,0.999880,0.564138,0.632043,-1.081770,0.999770,0.582141,0.632065,...,-0.008424,0.0,0.626426,0.622005,0.015757,0.0,0.632255,0.613136,0.016358,0.0
4,0.540633,0.729595,-1.241273,0.999880,0.564175,0.632645,-1.175990,0.999773,0.582163,0.633154,...,-0.009060,0.0,0.621938,0.626272,0.013444,0.0,0.627654,0.617958,0.013918,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,0.496425,0.593430,-0.669274,0.999830,0.511742,0.534296,-0.619719,0.999675,0.528450,0.536099,...,0.003401,0.0,0.574601,0.573254,0.021600,0.0,0.578985,0.565812,0.022490,0.0
800,0.503425,0.606916,-0.690603,0.999844,0.520325,0.543547,-0.640947,0.999700,0.537847,0.544827,...,0.003378,0.0,0.583959,0.591348,0.021250,0.0,0.588272,0.584137,0.022080,0.0
801,0.514170,0.616959,-0.753288,0.999857,0.532409,0.551677,-0.693692,0.999724,0.549591,0.552718,...,0.003247,0.0,0.590335,0.594324,0.021593,0.0,0.594581,0.587967,0.022282,0.0
802,0.517924,0.626695,-0.749026,0.999867,0.536497,0.558733,-0.695481,0.999746,0.553766,0.558982,...,0.002374,0.0,0.592238,0.595384,0.021352,0.0,0.596520,0.588399,0.022088,0.0


In [36]:
y

0           Happy
1           Happy
2           Happy
3           Happy
4           Happy
          ...    
799    Victorious
800    Victorious
801    Victorious
802    Victorious
803    Victorious
Name: class, Length: 804, dtype: object

In [37]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [39]:
display(X_train.shape)
display(y_train.shape)

display(X_test.shape)
display(y_test.shape)

(643, 2004)

(643,)

(161, 2004)

(161,)

### 3.2 Train machine learning classification model

In [47]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from tqdm import tqdm

In [42]:
pipelines = {
    'lr' : make_pipeline(StandardScaler(), LogisticRegression()),
    'rf' : make_pipeline(StandardScaler(), RandomForestClassifier()),
    'rc' : make_pipeline(StandardScaler(), RidgeClassifier()),
    'gb' : make_pipeline(StandardScaler(), GradientBoostingClassifier())
}

In [45]:
list(pipelines.values())[0]

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [48]:
fit_models = {}

for algo, pipeline in tqdm(pipelines.items()):
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

  0%|          | 0/4 [00:00<?, ?it/s]/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
100%|██████████| 4/4 [00:45<00:00, 11.38s/it]


In [49]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [52]:
fit_models['rc'].predict(X_test)

array(['Happy', 'Victorious', 'Happy', 'Sad', 'Sad', 'Victorious',
       'Victorious', 'Sad', 'Happy', 'Victorious', 'Happy', 'Victorious',
       'Happy', 'Victorious', 'Sad', 'Sad', 'Sad', 'Happy', 'Sad', 'Sad',
       'Happy', 'Happy', 'Sad', 'Victorious', 'Happy', 'Happy', 'Sad',
       'Victorious', 'Happy', 'Happy', 'Victorious', 'Sad', 'Happy',
       'Sad', 'Victorious', 'Happy', 'Victorious', 'Sad', 'Happy', 'Sad',
       'Happy', 'Victorious', 'Victorious', 'Happy', 'Victorious',
       'Happy', 'Sad', 'Sad', 'Sad', 'Victorious', 'Sad', 'Sad', 'Happy',
       'Victorious', 'Sad', 'Victorious', 'Victorious', 'Happy',
       'Victorious', 'Victorious', 'Victorious', 'Victorious', 'Happy',
       'Happy', 'Victorious', 'Happy', 'Victorious', 'Happy', 'Happy',
       'Happy', 'Sad', 'Victorious', 'Happy', 'Victorious', 'Happy',
       'Happy', 'Victorious', 'Victorious', 'Sad', 'Victorious', 'Happy',
       'Victorious', 'Victorious', 'Happy', 'Happy', 'Sad', 'Sad', 'Sad',
     

### 3.3 Evaluate and Serialize model

In [53]:
from sklearn.metrics import accuracy_score # calculate accuracy
import pickle # 

In [54]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 0.9875776397515528
rf 0.9875776397515528
rc 0.9937888198757764
gb 0.9813664596273292


In [63]:
fit_models['rc'].predict(X_test)

array(['Happy', 'Victorious', 'Happy', 'Sad', 'Sad', 'Victorious',
       'Victorious', 'Sad', 'Happy', 'Victorious', 'Happy', 'Victorious',
       'Happy', 'Victorious', 'Sad', 'Sad', 'Sad', 'Happy', 'Sad', 'Sad',
       'Happy', 'Happy', 'Sad', 'Victorious', 'Happy', 'Happy', 'Sad',
       'Victorious', 'Happy', 'Happy', 'Victorious', 'Sad', 'Happy',
       'Sad', 'Victorious', 'Happy', 'Victorious', 'Sad', 'Happy', 'Sad',
       'Happy', 'Victorious', 'Victorious', 'Happy', 'Victorious',
       'Happy', 'Sad', 'Sad', 'Sad', 'Victorious', 'Sad', 'Sad', 'Happy',
       'Victorious', 'Sad', 'Victorious', 'Victorious', 'Happy',
       'Victorious', 'Victorious', 'Victorious', 'Victorious', 'Happy',
       'Happy', 'Victorious', 'Happy', 'Victorious', 'Happy', 'Happy',
       'Happy', 'Sad', 'Victorious', 'Happy', 'Victorious', 'Happy',
       'Happy', 'Victorious', 'Victorious', 'Sad', 'Victorious', 'Happy',
       'Victorious', 'Victorious', 'Happy', 'Happy', 'Sad', 'Sad', 'Sad',
     

In [56]:
y_test

192         Happy
717    Victorious
168         Happy
522           Sad
536           Sad
          ...    
658    Victorious
356           Sad
525           Sad
748    Victorious
517           Sad
Name: class, Length: 161, dtype: object

In [69]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# 4. Make Detections with Model

w

In [70]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [71]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [73]:
# feed from webcam - connecting to webcam 
cap = cv2.VideoCapture(0)

cv2.startWindowThread()
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened(): # check wether or not webcam is open
        ret, frame = cap.read() # frame 변수에 webcam 으로 부터 받아온 이미지를 저장한다.
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections - holistic model에 detection 하는 process 함수에 image 전달
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Export coordinates
        try:
            # extract landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmarks.x, landmarks.y, landmarks.z, landmarks.visibility] for landmarks in pose]).flatten())
            
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmarks.x, landmarks.y, landmarks.z, landmarks.visibility] for landmarks in face]).flatten())
            
            # concatenate pose and face landmarks
            row = pose_row + face_row
            
            # row.insert(0, class_name)
            
            # with open('coords.csv', mode='a', newline='') as f :
            #     csv_writer = csv.writer(f, delimiter=',' , quotechar='"', quoting=csv.QUOTE_MINIMAL)
            #     csv_writer.writerow(row)
            
            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            # print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        except:
            pass
        
        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid f

-1